In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.util.remote import Remote
from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.operators.sampling.rnd import FloatRandomSampling,IntegerRandomSampling
from pymoo.operators.repair.rounding import RoundingRepair
import time
import matplotlib.pyplot as plt
import math
import copy
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor


In [2]:
## Função para calcular o retorno logarítmico
def log_return(series):
    return np.log(series / series.shift(1))


In [3]:

# Definindo uma função bagging, que recebe um dataframe e retorna um vetor de dataframes.
def Bagging(df, n, gamma):
    df_bagged = []
    for i in range(gamma):
        aux = df.sample(n = n, random_state = i)
        aux = aux.sort_index()
        df_bagged.append(aux)
    return df_bagged


In [4]:

def Add(y_hat, data_complete, ind, crypto):
    data_complete[crypto].at[ind, 'exp_value'] = y_hat


In [5]:

# Criptomoedas que vamos escolher para nosso portifólio
cryptos = [
    'BTC-USD', 'ETH-USD','LTC-USD', 'ADA-USD',
    'DOT-USD', 'LINK-USD','SOL-USD',
    'TRX-USD'
]


In [6]:
data = {}
for crypto in cryptos:
    # Baixar dados e calcular retorno logarítmico
    df = yf.download(crypto, start='2020-08-21', end=pd.to_datetime("today").strftime("%Y-%m-%d"))
    df['Return'] = log_return(df['Close']).shift(-2)

    # Construir DataFrame final, pegando o retorno do dia anterior e o atual
    df_final = pd.DataFrame({
        'Crypto_Return_Day_1': df['Return'].shift(1),
        'Crypto_Return_Today': df['Return']
    }).dropna()

    data[crypto] = df_final[['Crypto_Return_Today']]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:

data_complete = {}
for crypto in cryptos:
    # Baixar dados e calcular retorno logarítmico
    df = yf.download(crypto, start='2020-08-21', end=pd.to_datetime("today").strftime("%Y-%m-%d"))
    df['Return'] = log_return(df['Close']).shift(-2)

    # Construir DataFrame final, pegando o retorno do dia anterior e o atual
    df_final = pd.DataFrame({
        'Crypto_Return_Day_1': df['Return'].shift(1),
        'Crypto_Return_Today': df['Return']
    }).dropna()

    # Adicionar a coluna 'exp_value' com valores NaN
    df_final['exp_value'] = np.nan

    # Armazenar no dicionário
    data_complete[crypto] = df_final[['Crypto_Return_Today', 'exp_value']]



[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
W = 100      ####     HIPER PARAMETRO     #####

# Criando a função First Transformation, que recebe uma tabela contendo os retornos de todos os dias dos ultimos X anos e retorno um vetor, que cada elemento
# é uma tabela do retorno de W+1 dias.
def FirstTransform(df, W):
    vet = []
    Linhas, Colunas = df.shape
    for i in range(Linhas, W+1, -1):
        vet.append(df.iloc[(i-W-1):i])
    return vet


In [9]:

for crypto in cryptos:
    data[crypto] = FirstTransform(data[crypto], W)


In [10]:

test_target = {}
for crypto in cryptos:
    row = []
    for i in range(len(data[crypto])):
        row.append((pd.DataFrame(data[crypto][i].iloc[-1])).T)
        data[crypto][i] = data[crypto][i].drop(data[crypto][i].index[-1])
    test_target[crypto] = row


In [13]:
# Função do modelo LSTM otimizado
def Model(df):
    # Normalizando o dataframe
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)
    df_array = np.array(df_scaled)

    # Transformando no formato LSTM usando vetorização
    timesteps = 15
    X_lstm = np.array([df_array[i-timesteps:i, :] for i in range(timesteps, len(df_array))])
    y_lstm = df_array[timesteps:]

    # Separar X_train e y_train (todos exceto o último elemento)
    X_train = X_lstm[:-1]
    y_train = y_lstm[:-1]

    # X_test e y_test (último elemento)
    X_test = X_lstm[-1:]
    y_test = y_lstm[-1:]

    # Definir o modelo
    model = Sequential()
    model.add(LSTM(units=15, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Treinar o modelo (ajustar epochs dinamicamente com early stopping)
    early_stopping = EarlyStopping(monitor='loss', patience=3)
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, callbacks=[early_stopping])

    # Fazer a previsão
    y_hat_scaled = model.predict(X_test)
    y_hat = scaler.inverse_transform(y_hat_scaled)[0][0]

    return y_hat

In [12]:
def Add(y_hat, data_complete, ind, crypto):
    data_complete[crypto].at[ind, 'exp_value'] = y_hat

In [13]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Usando GPU:", physical_devices[0])
else:
    print("Usando CPU")


Usando CPU


In [15]:
# Função que processa cada criptomoeda (paralelismo será aplicado aqui)
def process_crypto(crypto, data, test_target, data_complete, D):
    for day in range(0, D):
        df_train = data[crypto][day].copy()  # df é uma tabela de 300 linhas e uma coluna
        df_test = test_target[crypto][day].copy()  # df_test é uma tabela com 1 linha e uma coluna
        df = pd.concat([df_train, df_test], ignore_index=True)  # unindo as duas tabelas

        y_hat = Model(df)  # chamar o modelo para previsão
        Add(y_hat, data_complete, df_test.index[0], crypto)  # adicionar o resultado na tabela final

# Número de dias que vamos testar o modelo
D = 60

# Criando o executor para rodar as criptomoedas em paralelo
with ThreadPoolExecutor() as executor:
    # Executa o processo para cada criptomoeda simultaneamente
    futures = [
        executor.submit(process_crypto, crypto, data, test_target, data_complete, D)
        for crypto in cryptos
    ]

    # Aguardar a conclusão de todas as threads
    for future in futures:
        future.result()

print("Processamento em paralelo finalizado.")

KeyboardInterrupt: 

In [32]:
data_complete['ETH-USD']

,Crypto_Return_Today,exp_value
Date,,
2020-08-22,0.041930,NaN
2020-08-23,-0.060975,NaN
2020-08-24,0.006399,NaN
2020-08-25,-0.009969,NaN
2020-08-26,0.034022,NaN
...,...,...
2024-10-08,0.006555,0.005819
2024-10-09,0.021848,-0.029071
2024-10-10,0.016288,-0.020197


In [33]:
for crypto in cryptos:
    # Calcular a diferença entre Crypto_Return_Today e exp_value
    # Calcular o desvio padrão do Crypto_Return_Today nos últimos 30 dias
    data_complete[crypto]['volatility'] = data_complete[crypto]['Crypto_Return_Today'].rolling(window=30).std()

In [34]:
for crypto in cryptos:
    # Define o nome do arquivo como o nome da criptomoeda seguido de .csv
    filename = f"7{crypto}.csv"
    
    # Salva o DataFrame data_complete[crypto] no arquivo .csv
    data_complete[crypto].to_csv(filename, index=False)

    print(f"Arquivo {filename} salvo com sucesso!")

Arquivo 7BTC-USD.csv salvo com sucesso!
Arquivo 7ETH-USD.csv salvo com sucesso!
Arquivo 7LTC-USD.csv salvo com sucesso!
Arquivo 7ADA-USD.csv salvo com sucesso!
Arquivo 7DOT-USD.csv salvo com sucesso!
Arquivo 7LINK-USD.csv salvo com sucesso!
Arquivo 7SOL-USD.csv salvo com sucesso!
Arquivo 7TRX-USD.csv salvo com sucesso!


In [35]:
data_complete['ETH-USD']

,Crypto_Return_Today,exp_value,volatility
Date,,,
2020-08-22,0.041930,NaN,NaN
2020-08-23,-0.060975,NaN,NaN
2020-08-24,0.006399,NaN,NaN
2020-08-25,-0.009969,NaN,NaN
2020-08-26,0.034022,NaN,NaN
...,...,...,...
2024-10-08,0.006555,0.005819,0.024612
2024-10-09,0.021848,-0.029071,0.024598
2024-10-10,0.016288,-0.020197,0.024709
